# <div align='center'> GPU内存资源调度 </div>

In [ ]:
```
                                                                         
              客户                    业务                      框架     
                                                                         
                |                      |                         |       
                |  REQ: resource       |                         |       
                | -------------------> |                         |       
                |  RES:                |                         |       
                | <------------------- |                         |       
                |                      |                         |       
       check ---+                      |                         |       
              / | \ Y                  |                         |       
          N  /  |  \                   |                         |       
        +---+   |  ------------------> |   RESTAPI: train        |       
        | o |   |                      | ----------------------> |       
        |   |   |                      |   RET: current resource |       
        |   |   |                      | <---------------------- |       
        |   |   |          record: mem |                         |       
   W  Q |   |   |                      |       Out of mem        |       
      U | . |   |         mem + 500M   | <---------------------- |       
      E | . |<-------------------------|                         |       
      U |   |   |                      |                         |       
      E |   |   |                      |                         |       
        |   |   |                      |                         |       
        | o |   |                      |                         |       
        +---+   |                      |                         |       
          |     |                      |                         |       
          |     |   POP                |    RESTAPI: train       |       
          +--------------------------->| ----------------------> |       
                |                      |     OK                  |       
                |                      | <---------------------- |       
        +---+   |   PUSH               |                         |       
        | o |<-------------------------|                         |       
        |   |   |                      |                         |       
        |   |   |                      |                         |       
        |   |   |                      |                         |       
   R  Q |   |   |                      |                         |       
      U |   |   |                      |                         |       
      E |   |   |                      |                         |       
      U |   |   |                      |                         |       
      E |   |   |                      |                         |       
        |   |   |                      |                         |       
        |   |   |                      |                         |       
        +---+   |                      |                         |       
                |                      |                         |       
```

## 模块提供

1. 框架(C): 任务启动时返回当前系统(GPU)的内存使用情况

2. 业务(B): 维护两个任务队列(Running Queue, Waiting Queue), 及**任务类型资源需求矩阵**

3. 用户(A):

## 流程

1. 用户(A)启动一次训练, 点击"开始", 业务(B)先从**任务资源需求矩阵**中获取该次训练大概需要使用多少内存, 并将该值和训练参数一并传到框架(C)端


2. 框架(C)接收到任务请求后, 先获取一下当前系统剩余的内存(GPU)是否能够满足请求中的内存需求(步骤1传的参数), 如果不能满足, 直接返回错误信息, 错误信息中包含当前系统中剩余的内存


3. 业务(B)收到这个错误后, 将该任务从running态转移到waiting态, 且业务(B)应该会有3中情况需要并更队列: (1) 框架(C)报错, 任务退出,释放内存(被动方式) (2) 框架(C)任务正常结束或者用户(A)手动停止训练, 释放内存(被动方式) (3) 业务定时查询框架(C)资源信息(主动方式)


4. 框架(C)任务训练过程中记录本次训练需要的**最大**内存资源信息, 并且任务结束后将该信息作为metrics中的一项返回给业务(B)


5. 业务(B)收到步骤4中的metrics后, 更新**任务类型资源需求矩阵**


6. 有种特殊情况, **任务资源需求矩阵**里面的记录的资源信息低估了任务实际所需的资源(GPU内存), 这种情况下, 启动任务, 框架(C)如果发下当前系统的剩余资源能够运行这个任务, 会返回成功(同时也会携带当前系统剩余mem), 当任务container运行一段时间后, 由于低估了资源使用, 实际使用的内存正好超过了系统的剩余资源, 则会报**out of mem**的错误信息.


7. 如果步骤6发生, 接收到错误后, 把该任务从running转到waiting, 同时更新**任务类型资源需求矩阵**(用启动训练时返回的mem 加上一个增量比如500MB)
